In [ ]:
import torch
import torchvision.datasets
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Hyperparameters
epochs = 10
batch_size = 100


# Data Normalization
normal = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

file_path = "File:///users/admins/downloads/CIFAR-10.ipynb"

training_data = torchvision.datasets.CIFAR10(root = file_path, train = True,transform = normal,download = True)
val_data = torchvision.datasets.CIFAR10(root = file_path, train= False, transform = normal)

train_loader = DataLoader(dataset = training_data, batch_size = batch_size, shuffle = True)
val_loader  = DataLoader(dataset = val_data, batch_size = batch_size, shuffle = False)

# Helper Methods

def create_conv_layer(in_channels,out_channels,kernel_size,stride,padding,num_layers):
    return nn.Sequential(nn.Conv2d(in_channels,out_channels,kernel_size = kernel_size, stride = stride, padding = padding,),nn.ReLU(),nn.MaxPool2d(kernel_size = 2, stride  = 2),nn.BatchNorm2d(out_channels))

def create_linear_layer(image_size,in_channels,out_channels,kernel_size,stride,padding,num_conv_layers):
  for conv_layer in range(num_conv_layers):
    image_size = (image_size + 2*padding - kernel_size)/stride + 1
    image_size = image_size/2
  input_size = int(image_size * image_size * in_channels)
  return nn.Linear(input_size,out_channels)

# Create the model
class Conv2D(nn.Module):
  def __init__(self):
    super(Conv2D,self).__init__()
    self.conv1 = create_conv_layer(3,10,3,1,1,1)
    self.conv2 = create_conv_layer(10,20,3,1,1,1)
    self.dropout_layer = nn.Dropout()
    self.fc1 = create_linear_layer(32,20,250,3,1,1,2)
    self.fc2 = nn.Linear(250,100)
    self.fc3 = nn.Linear(100,10)
  
  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.reshape(len(x),-1)
    x = self.dropout_layer(x)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    return x

model = Conv2D()


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
train_correct = 0
val_correct = 0

for epoch in range(epochs):
  for case, (image,label) in enumerate(train_loader):
    pred = model(image)
    loss = loss_fn(pred,label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    _, batch_prediction  = pred.max(1)
    train_correct = (batch_prediction == label).sum().item()
    if((case+1)%100 == 0):
      print("Epoch: {} Batch: {} Batch Accuracy {:.4%} Batch Cost: {:.4f}".format(epoch+1,int(case/100)+1,(train_correct/batch_size),loss))

model.eval()
for test_case, (image,label) in enumerate(val_loader):
  pred = model(image)
  _, val_pred = pred.max(1)
  val_correct += (val_pred == label).sum().item()
print("Validation Accuracy: {:.4%}".format(val_correct/len(val_data)))
  
  






Files already downloaded and verified
Epoch: 1 Batch: 1 Batch Accuracy 45.0000% Batch Cost: 1.6266
Epoch: 1 Batch: 2 Batch Accuracy 49.0000% Batch Cost: 1.3304
Epoch: 1 Batch: 3 Batch Accuracy 46.0000% Batch Cost: 1.4741
Epoch: 1 Batch: 4 Batch Accuracy 54.0000% Batch Cost: 1.1729
Epoch: 1 Batch: 5 Batch Accuracy 64.0000% Batch Cost: 1.1213
Epoch: 2 Batch: 1 Batch Accuracy 57.0000% Batch Cost: 1.2413
Epoch: 2 Batch: 2 Batch Accuracy 60.0000% Batch Cost: 1.1263
Epoch: 2 Batch: 3 Batch Accuracy 54.0000% Batch Cost: 1.1837
Epoch: 2 Batch: 4 Batch Accuracy 61.0000% Batch Cost: 1.0221
Epoch: 2 Batch: 5 Batch Accuracy 68.0000% Batch Cost: 1.1074
Epoch: 3 Batch: 1 Batch Accuracy 63.0000% Batch Cost: 1.1058
Epoch: 3 Batch: 2 Batch Accuracy 61.0000% Batch Cost: 1.0880
Epoch: 3 Batch: 3 Batch Accuracy 64.0000% Batch Cost: 1.0307
Epoch: 3 Batch: 4 Batch Accuracy 58.0000% Batch Cost: 1.1689
Epoch: 3 Batch: 5 Batch Accuracy 57.0000% Batch Cost: 1.2890
Epoch: 4 Batch: 1 Batch Accuracy 55.0000% Batch